In [3]:
# Fine-tuning kodu (train.py gibi düşünebilirsin)

import os
import pandas as pd
from PIL import Image
from torch.utils.data import Dataset
from transformers import BlipProcessor, BlipForConditionalGeneration, TrainingArguments, Trainer
import torch

2025-05-22 12:04:00.128742: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747915440.323496      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747915440.384033      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# BLIP processor ve model
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [5]:
class ImageCaptionDataset(Dataset):
    def __init__(self, csv_file, image_folder, processor):
        self.data = pd.read_csv(csv_file)
        self.image_folder = image_folder
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = str(self.data.iloc[idx]['image_id']) + ".jpg"
        img_path = os.path.join(self.image_folder, img_name)
        caption = self.data.iloc[idx]['caption']
        image = Image.open(img_path).convert("RGB")
    
        inputs = self.processor(
            images=image,
            text=caption,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=128
        )
        
        # loss için labels eklenmeli
        inputs['labels'] = inputs['input_ids']
    
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs

In [6]:
# Kaggle dataset klasörleri
train_csv = '/kaggle/input/obss-intern-competition-2025/train.csv'
train_images = '/kaggle/input/obss-intern-competition-2025/train/train/'

train_dataset = ImageCaptionDataset(train_csv, train_images, processor)

# Eğitim argümanları
training_args = TrainingArguments(
    output_dir="/kaggle/working/blip-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_dir='/kaggle/working/logs',
    save_total_limit=1,
    save_steps=500,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

In [7]:
# Eğitimi başlat
trainer.train()

Step,Training Loss
10,9.484900
20,7.175400
30,6.033600
40,5.033000
50,3.998400
60,2.970200
70,2.048300
80,1.263000
90,0.870600
100,0.690900


TrainOutput(global_step=16026, training_loss=0.29926554897550667, metrics={'train_runtime': 7798.8413, 'train_samples_per_second': 8.219, 'train_steps_per_second': 2.055, 'total_flos': 3.803904953127626e+19, 'train_loss': 0.29926554897550667, 'epoch': 3.0})

In [10]:
# Modeli kaydet
model.save_pretrained("/kaggle/working/blip-finetuned")
processor.save_pretrained("/kaggle/working/blip-finetuned")

[]

In [11]:
!zip -r /kaggle/working/blip-finetuned.zip /kaggle/working/blip-finetuned

  adding: kaggle/working/blip-finetuned/ (stored 0%)
  adding: kaggle/working/blip-finetuned/preprocessor_config.json (deflated 48%)
  adding: kaggle/working/blip-finetuned/model.safetensors

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 (deflated 7%)
  adding: kaggle/working/blip-finetuned/config.json (deflated 67%)
  adding: kaggle/working/blip-finetuned/special_tokens_map.json (deflated 80%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/ (stored 0%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/training_args.bin (deflated 52%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/rng_state.pth (deflated 25%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/model.safetensors (deflated 7%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/config.json (deflated 67%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/generation_config.json (deflated 29%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/optimizer.pt (deflated 14%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/scheduler.pt (deflated 55%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/scaler.pt (deflated 60%)
  adding: kaggle/working/blip-finetuned/checkpoint-16026/trainer_state.js

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm

# Processor ana dizinden yüklenecek
caption_processor = BlipProcessor.from_pretrained("/kaggle/working/blip-finetuned")

# Model checkpoint dizininden yüklenecek
caption_model = BlipForConditionalGeneration.from_pretrained("/kaggle/working/blip-finetuned/checkpoint-16026")

# Embedding modeli
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Test dosyası ve klasörü
test_csv = '/kaggle/input/obss-intern-competition-2025/test.csv'
test_images = '/kaggle/input/obss-intern-competition-2025/test/test/'

test_df = pd.read_csv(test_csv)
test_df["image_id"] = test_df["image_id"].astype(str)  # Burada stringe dönüştür

captions = []

print("Caption üretimi başlıyor...")
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = os.path.join(test_images, str(row["image_id"]) + ".jpg")
    image = Image.open(image_path).convert("RGB")

    inputs = caption_processor(image, return_tensors="pt")
    out = caption_model.generate(**inputs)
    caption = caption_processor.decode(out[0], skip_special_tokens=True)

    captions.append(caption)

# Submission dosyası oluştur
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Captionlar submission.csv dosyasına kaydedildi.")

Caption üretimi başlıyor...


100%|██████████| 3771/3771 [3:00:39<00:00,  2.87s/it]  

Captionlar submission.csv dosyasına kaydedildi.
Embedding hesaplama başlıyor...


Batches: 0it [00:00, ?it/s]

Batches: 0it [00:00, ?it/s]

FGD Score: nan


/usr/local/lib/python3.11/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


100%|██████████| 3771/3771 [3:00:39<00:00,  2.87s/it]  

submission.csv

Skor: 0.17392